# Real Time Object Detection with TensorFlow Detection Model

https://towardsdatascience.com/real-time-object-detection-with-tensorflow-detection-model-e7fd20421d5d
https://github.com/ElephantHunters/Real_time_object_detection_using_tensorflow/blob/master/object_detection_tutorial_Webcam_1.ipynb
<br/>Object detection with tensorflow
<br/>
https://github.com/tensorflow/models/tree/master/research/object_detection
<br/>Documentation models:
<br/>
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
<br/>
Manuel Robalinho - 29-10-2018

## Installation for object-detection
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

<b>Download Tensorflow repository:</b>
<br/>conda install tensorflow

<b>Clone Tensorflow and Tensorflow models repositories:</b>
<br/>git clone https://github.com/tensorflow/tensorflow
<br/>cd tensorflow
<br/>git clone https://github.com/tensorflow/models

<b>Install Object Detection API:</b>
<br/> guide:https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

<br/>sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
<br/>conda install protobuf-compiler
<br/>conda install --user Cython
<br/>conda install --user contextlib2
<br/>conda install --user jupyter
<br/>conda install --user matplotlib

<b>Install COCO API</b>
<br/>Change <path_to_tensorflow> to path to your Tensorflow repository.
<br/>git clone https://github.com/cocodataset/cocoapi.git
<br/>cd cocoapi/PythonAPI
<br/>make
<br/>cp -r pycocotools <path_to_tensorflow>/models/research/

<b>Install Protoc</b>
<br/>You can download it here: https://github.com/google/protobuf/releases
<br/># From tensorflow/models/research/
<br/>protoc object_detection/protos/*.proto --python_out=.

<b>Install object detection</b>
<br/>If you want to run object detection from any directory, I guess you want, you will need to add <br/>the path to your ~/.bashrc file. Obviously change <path_to_tensorflow> , replace ~with $HOME.

<br/>export PYTHONPATH=$PYTHONPATH:<path_to_tensorflow>/models/research/: path_to_tensorflow>/models/research/slim

#### Need install MAKE on:
    
http://stat545.com/automation02_windows.html
    

## Import Libraries

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
print('TensorFlow Version:',tf.__version__)
if tf.__version__ < '1.04.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

TensorFlow Version: 1.8.0


## Environment setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
print('We are in the path: ',os.getcwd())

We are in the path:  C:\Users\Lenovo\Object_Detection_Real_Time


## Object detection imports
Here are the imports from the object detection module.

In [3]:
import sys, os
# Add new path
sys.path.append('./utils')
# View paths
sys.path

['',
 'C:\\ProgramData\\Anaconda3\\python36.zip',
 'C:\\ProgramData\\Anaconda3\\DLLs',
 'C:\\ProgramData\\Anaconda3\\lib',
 'C:\\ProgramData\\Anaconda3',
 'C:\\Users\\Lenovo\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Sphinx-1.5.6-py3.6.egg',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\object_detection-0.1-py3.6.egg',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Lenovo\\.ipython',
 './utils']

In [4]:
# My utils is in directory: models\object_detection\utils (download from Tensorflow)
from models.object_detection.utils import label_map_util

from models.object_detection.utils import visualization_utils as vis_util

## Model preparation
### Variables
Any model exported using the export_inference_graph.py tool can be loaded here simply by changing PATH_TO_CKPT to point to a new .pb file.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
my_model_detection = 'frozen_inference_graph.pb'
PATH_TO_CKPT = MODEL_NAME + '/' + my_model_detection

# List of the strings (mscoco_label_map.pbtxt) that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('models\object_detection\data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [6]:
# Confirm Model Name
MODEL_NAME

'ssd_mobilenet_v1_coco_2017_11_17'

In [7]:
# Confirm path to model detection
PATH_TO_CKPT

'ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'

## Download Model

In [8]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if my_model_detection in file_name:
        tar_file.extract(file, os.getcwd())

In [9]:
# Confirm if loaded Model run correctly
print(tar_file)

## Load a (frozen) Tensorflow model into memory.

In [10]:
PATH_TO_CKPT

'ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'

In [11]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts 5, we know that this corresponds to airplane. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [12]:
# Confirm labels path
PATH_TO_LABELS

'models\\object_detection\\data\\mscoco_label_map.pbtxt'

In [13]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [14]:
print(label_map)

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}
item {
  name: "/m/0k4j"
  id: 3
  display_name: "car"
}
item {
  name: "/m/04_sv"
  id: 4
  display_name: "motorcycle"
}
item {
  name: "/m/05czz6l"
  id: 5
  display_name: "airplane"
}
item {
  name: "/m/01bjv"
  id: 6
  display_name: "bus"
}
item {
  name: "/m/07jdr"
  id: 7
  display_name: "train"
}
item {
  name: "/m/07r04"
  id: 8
  display_name: "truck"
}
item {
  name: "/m/019jd"
  id: 9
  display_name: "boat"
}
item {
  name: "/m/015qff"
  id: 10
  display_name: "traffic light"
}
item {
  name: "/m/01pns0"
  id: 11
  display_name: "fire hydrant"
}
item {
  name: "/m/02pv19"
  id: 13
  display_name: "stop sign"
}
item {
  name: "/m/015qbp"
  id: 14
  display_name: "parking meter"
}
item {
  name: "/m/0cvnqh"
  id: 15
  display_name: "bench"
}
item {
  name: "/m/015p6"
  id: 16
  display_name: "bird"
}
item {
  name: "/m/01yrx"
  id: 17
  display_name: "cat

In [15]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

In [34]:
import cv2

cap = cv2.VideoCapture(0) # 0 stands for very first webcam attach

filename="output\VideoCap1.avi" #[place were i stored my output file]

codec = cv2.VideoWriter_fourcc(*'MJPG')  # AVI
#codec = cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code

framerate=2
resolution=(640,480)
    
VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)


In [35]:
#Confirm Video
VideoFileOutput

<VideoWriter 0000015CCF87DE50>

In [36]:
with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        ret=True
            
        while (ret):
        
            ret, image_np=cap.read() 

            # Define input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores  = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections    = detection_graph.get_tensor_by_name('num_detections:0')

              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
              # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
              # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=8)

            VideoFileOutput.write(image_np)
            cv2.imshow('MRobalinho - Tensorflow - live_detection',image_np)
            xkey = cv2.waitKey(1) & 0XFF  # wait 1 seconds
            #print(xkey) verify Key q to exit
            if xkey == ord('q'):  # Key q to Exit (key q)
                print('Key q Pressed - Exit')
                VideoFileOutput.release()
                break

cap.release()
cv2.destroyAllWindows()

Key q Pressed - Exit


In [37]:
tf.Session(graph=detection_graph)